# Fixing triangular meshes with trimesh

Consider: 
* The function `txttoarray` will only work for txt files inside the folder this notebook is in.
* Mesh will be made watertight by filling holes, deleting repeted vertices and faces, and by merging vertices that are close together. This distance can be toggled through the variable `merge_tolerance`.

In [1]:
%reset -f
import time
import numpy as np
import trimesh


def txttoarray(filename):
    file = open(filename, "r").read().split("\n")
    datos=[]
    for i in range(len(file)):
        datos.append(file[i].split())
    return(datos)

######################### #########################
######################### #########################

toc = time.time()
    
# Replace "surf_d02_stern.face" and "vert" with filename (txt extention)
face=txttoarray("Meshing_1smd/1smd_d08.face")
face=[i for i in face if i!=[]]

vert=txttoarray("Meshing_1smd/1smd_d08.vert")
vert=[i for i in vert if i!=[]]

vert_txt=np.float_(vert) ; vert_norm=vert_txt[:,3:6] ; vert=vert_txt[:,:3]
face=np.int_(face); face= face[:,:3]-1

######################### #########################
######################### #########################
    
mesh = trimesh.Trimesh(vertices=vert,faces=face,vertex_normals=vert_norm,validate=True,process=True)
mesh.fill_holes()
mesh.process()

while not mesh.is_watertight:
    merge_tolerance = 0.005
    needy_faces     = trimesh.repair.broken_faces(mesh)
    for vert_nf in mesh.faces[needy_faces]:
        for nf in vert_nf:
            for c,check in enumerate(np.linalg.norm(mesh.vertices[vert_nf]-mesh.vertices[nf],axis=1)):
                if (check<merge_tolerance) & (0<check):
                    mesh.vertices[nf]=mesh.vertices[vert_nf[c]]
                    
mesh.fill_holes()
mesh.process()
            
                
tic = time.time()
print("Total time:",round(tic-toc,3),"[s]")

Total time: 2.11 [s]


In [2]:
print(mesh)

<trimesh.Trimesh(vertices.shape=(66850, 3), faces.shape=(133620, 3))>


In [3]:
mesh_split = mesh.split()
print(mesh)
print(mesh_split)
print("len de mesh_split: ",len(mesh_split))

<trimesh.Trimesh(vertices.shape=(66850, 3), faces.shape=(133620, 3))>
[<trimesh.Trimesh(vertices.shape=(62500, 3), faces.shape=(125016, 3))>
 <trimesh.Trimesh(vertices.shape=(756, 3), faces.shape=(1508, 3))>
 <trimesh.Trimesh(vertices.shape=(352, 3), faces.shape=(700, 3))>
 <trimesh.Trimesh(vertices.shape=(204, 3), faces.shape=(404, 3))>
 <trimesh.Trimesh(vertices.shape=(142, 3), faces.shape=(280, 3))>
 <trimesh.Trimesh(vertices.shape=(264, 3), faces.shape=(524, 3))>
 <trimesh.Trimesh(vertices.shape=(112, 3), faces.shape=(220, 3))>
 <trimesh.Trimesh(vertices.shape=(150, 3), faces.shape=(296, 3))>
 <trimesh.Trimesh(vertices.shape=(184, 3), faces.shape=(364, 3))>
 <trimesh.Trimesh(vertices.shape=(124, 3), faces.shape=(244, 3))>
 <trimesh.Trimesh(vertices.shape=(136, 3), faces.shape=(268, 3))>
 <trimesh.Trimesh(vertices.shape=(104, 3), faces.shape=(204, 3))>
 <trimesh.Trimesh(vertices.shape=(92, 3), faces.shape=(180, 3))>
 <trimesh.Trimesh(vertices.shape=(280, 3), faces.shape=(556, 3))>
 

In [4]:
mesh_ses = mesh_split[0]
print(mesh_ses)

<trimesh.Trimesh(vertices.shape=(126916, 3), faces.shape=(253844, 3))>


In [5]:
mesh_ses.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [6]:
mesh_ses_faces = mesh_ses.faces + 1
mesh_ses_vert = mesh_ses.vertices
print(mesh_ses_faces, mesh_ses_vert)

[[  2072    435   2173]
 [ 86005    888 110095]
 [ 26228  86005 110095]
 ..., 
 [  2807  87848  87289]
 [ 77387  28641  87866]
 [ 48195  63697  87858]] [[  0.     46.148  43.915]
 [ 30.72   92.432  21.205]
 [ 30.72   50.141  32.555]
 ..., 
 [ 20.963  84.577  19.194]
 [ 20.963  75.31    8.883]
 [ 20.963  72.699   9.037]]


In [7]:
np.savetxt("Meshing_1smd/1smd_d16_split.face",mesh_ses_faces,fmt="%i")
np.savetxt("Meshing_1smd/1smd_d16_split.vert",mesh_ses_vert)

In [8]:
#mesh: face - vert 1a0j
#d01: 7384 - 3690
#d02: 15684 - 7842
#d04: 32524 - 16260
#d08: 66580 - 33286
#d16: 135644 - 67816

#mesh: face - vert 1smd
#d01: 13896 - 6944
#d02: 31080 - 14650
#d04: 61240 - 30616
#d08: 125016 - 62500
#d16: 253844 - 135878